In [7]:
import geopandas as gpd

gpd.read_file('blocks (1).geojson')

,geometry


In [8]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan, basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

NotImplementedError: Multipolygon source is not supported yet

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

zones = {'1': 0.1,
         '2': 0.1,
         '3': 0.1,
         '4': 0.1,
         '5': 0.1,
         '6': 0.1,
         '7': 0.1,
         '8': 0.1, }
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_poly(zones, roads_width=50)
zones.explore()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely import MultiPolygon, Polygon

local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)


def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry, crs=gdf.crs).explode(ignore_index=True)
    max_dist = np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly


p = elasatic_wrap(multip)

In [ ]:
gpd.GeoDataFrame(geometry=[p.simplify(250)], crs=local_crs).explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner, gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'), rotation=0)
multiterritory = gp.transformed_poly
zones, roads = gp.poly2func2terr2block(gen_plan)

In [ ]:
zones.explore()

In [1]:
import geopandas as gpd
from app.gen_planner.python.src.utils.geom_utils import geometry_to_multilinestring
from shapely.ops import polygonize, unary_union


def territory_splitter(gdf_to_split: gpd.GeoDataFrame, splitters: gpd.GeoDataFrame,
                       transfer_splitters_data=False) -> gpd.GeoDataFrame:
    local_crs = gdf_to_split.estimate_utm_crs()
    gdf_to_split = gdf_to_split.to_crs(local_crs)
    splitters = splitters.to_crs(local_crs)
    lines_orig = gdf_to_split.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()
    lines_splitters = splitters.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()

    polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_splitters))),
                                crs=local_crs).clip(gdf_to_split.to_crs(local_crs),keep_geom_type=True)
    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    if transfer_splitters_data and splitters.geom_type.isin(['MultiPolygon', 'Polygon']).any():
        joined_ind = polygons_points.sjoin(splitters,how='inner',predicate='within').index.tolist()
        polygons['splitter_poly'] = polygons.index.isin(joined_ind)
    return polygons


local_crs = 32636

territory = gpd.read_file('project Шлиссельбург.geojson').to_crs(local_crs)
roads = gpd.read_file('roads_shlis.geojson').to_crs(local_crs)
# roads.geometry = roads.geometry.buffer(10, resolution=2)

res = territory_splitter(territory, roads,transfer_splitters_data=True)
res.explore()

Available workers count 16


In [4]:
from genplanner import GenPlanner

zones, roads = GenPlanner(res).poly2terr2block()

In [7]:
m1 = zones.explore(column='zone_name', tiles='CartoDB positron', cmap='Dark2', categorical=True)
res.explore(m=m1,style_kwds = {'fillOpacity':0.05})


In [7]:
roads

,geometry,roads_width
0,"LINESTRING (388862.449 6642960.048, 388919.353...",10
1,"LINESTRING (389239.052 6644252.285, 389238.595...",10
2,"LINESTRING (389238.595 6644252.938, 389124.614...",10
3,"LINESTRING (389124.614 6644286.68, 389096.097 ...",10
4,"LINESTRING (388836.27 6644218.922, 388834.487 ...",10
5,"LINESTRING (388834.487 6644222.998, 388418.064...",10
6,"LINESTRING (390775.837 6644450.141, 390721.048...",10
7,"LINESTRING (389035.228 6644487.805, 389049.622...",10
8,"LINESTRING (389049.622 6644550.946, 388831.327...",10
9,"LINESTRING (388412.105 6644382.078, 388418.064...",10


In [ ]:

from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr(func_zones.residential_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)